This another long puts strategy made by Brain and is a more simple approach than the other long put strat

In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime
import requests
from alpha_vantage.timeseries import TimeSeries

In [10]:
def calculate_linear_regression(prices):
    X = np.arange(len(prices)).reshape(-1, 1)
    y = prices.values
    model = LinearRegression().fit(X, y)
    return model.intercept_, model.coef_[0]

In [21]:
sp500_tickers = pd.read_csv('Data/sp500company.csv')['Symbol'].tolist()

positive_slope = {}
negative_slope = {}
put_options_data = {}

In [22]:
for ticker in sp500_tickers:
    data = yf.download(ticker, start='2023-01-01', end=datetime.datetime.today().strftime('%Y-%m-%d'))
    
    if not data.empty:
        intercept, slope = calculate_linear_regression(data['Close'])
        if slope > 0:
            positive_slope[ticker] = {
                'Ticker': ticker,
                'Intercept': intercept,
                'Slope': slope
            }
        else:
            negative_slope[ticker] = {
                'Ticker': ticker,
                'Intercept': intercept,
                'Slope': slope
            }

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [23]:
positive_slope_df = pd.DataFrame.from_dict(positive_slope, orient='index')
negative_slope_df = pd.DataFrame.from_dict(negative_slope, orient='index')

positive_slope_df.to_csv(r'C:\Users\bsung\OneDrive\Documents\GitHub\PortfolioProject\Data\ps_df.csv', index=False)
negative_slope_df.to_csv(r'C:\Users\bsung\OneDrive\Documents\GitHub\PortfolioProject\Data\ns_df.csv', index=False)

In [31]:
def get_put_options(ticker):
    url = f'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={ticker}&apikey=6WLY2DVR35CPCC7A'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'optionData' in data and 'putOptions' in data['optionData']:
            put_options = data['optionData']['putOptions']
            put_options_df = pd.DataFrame(put_options)
            print(f"Ticker: {ticker}")
            print(put_options_df.head(10))
            return put_options_df
    return None

In [32]:
tickers = positive_slope_df['Ticker'].tolist()

In [33]:
for ticker in tickers:
    put_options_df = get_put_options(ticker)
    if put_options_df is not None:
        put_options_data[ticker] = put_options_df

#runtime: 5 min 2.0s

In [ ]:
all_put_options_df = pd.concat(put_options_data.values(), keys=put_options_data.keys())

all_put_options_df.to_csv('all_put_options.csv')

print(all_put_options_df)

In [27]:
#put_options_df.to_csv(r'C:\Users\bsung\OneDrive\Documents\GitHub\PortfolioProject\Data\popt_df.csv', index=False)